# KeepYouSafe WebApp

In [ ]:
import cv2
import time
import imutils
import os
import numpy as np

from math import pow, sqrt
from imutils.video import FPS
from imutils.video import VideoStream

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

from flask import Flask, render_template, Response, request

from detecta import Detector



# iniciaciliza la aplicacion
app=Flask(__name__)

cam=VideoStream(src=0).start()

width=800   # pixeles imagen

modo=''  # modo camara

modelo=cv2.dnn.readNetFromCaffe('detector/SSD_MobileNet_prototxt.txt', 'detector/SSD_MobileNet.caffemodel')
    
net_cara=cv2.dnn.readNet('detector/deploy.prototxt', 'detector/res10_300x300_ssd_iter_140000.caffemodel')

# se carga el modelo detector de mascarillas
net_mascara=load_model('detector/detector_mascara.model')


detector=Detector(cam, width, modelo, net_cara, net_mascara)

    
@app.route('/tomar_foto', methods=['POST'])
def tomar_foto():
    foto=detector.foto
    cv2.imwrite('static/images/foto.png', foto)
    return Response(status=200)    
   
    

@app.route('/video')
def video():
    global modo

    if modo=='Mascara':
        return Response(detector.mascara(), mimetype='multipart/x-mixed-replace; boundary=frame')

    elif modo=='Distancia':
        return Response(detector.distancia(), mimetype='multipart/x-mixed-replace; boundary=frame')

    else:
        return Response(detector.camara(), mimetype='multipart/x-mixed-replace; boundary=frame')



    
@app.route('/', methods=['POST', 'GET'])
def main():
    if request.method=='POST':
        global modo
        modo=request.form['modo']
        return render_template('index.html', modo=modo)
    else:
        return render_template('index.html', modo=modo)
    
    
if __name__=='__main__':
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Aug/2020 20:38:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2020 20:38:37] "GET /video HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2020 20:38:39] "POST /tomar_foto HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2020 20:38:50] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2020 20:38:50] "GET /video HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2020 20:38:52] "POST /tomar_foto HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2020 20:39:00] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2020 20:39:00] "GET /video HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2020 20:39:02] "POST /tomar_foto HTTP/1.1" 200 -
